In [2]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.2'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [52.7 kB]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB

In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-04-03 20:43:07--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.47MB/s    in 0.2s    

2021-04-03 20:43:08 (5.47 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Toys_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Toys_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18778586| RDIJS7QYB6XNR|B00EDBY7X8|     122952789|Monopoly Junior B...|            Toys|          5|            0|          0|   N|                Y|          Five Stars|        Excellent!!!| 2015-08-31|
|         US|   24769659|R36ED1U38IELG8|B00D7JFOPC|     952062646|56 Pieces of Wood...|            Toys|          5|    

In [6]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
#Create new dataframe where the total_votes count is equal to or greater than 20
vote_count_df = vine_df.filter("total_votes >= 20")
vote_count_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| ROP6ITXO8K5V2|          5|           23|         27|   N|                Y|
|R3ND1LVU7AXCVF|          1|           21|         21|   N|                Y|
| R9I5FOLKU99RY|          5|           19|         20|   Y|                N|
|R1QS8AOD6HX3ED|          4|           59|         81|   N|                N|
|R3ED60RC69CCQ6|          5|           22|         23|   Y|                N|
|R2MDI1TP46VSYD|          3|            9|         21|   N|                Y|
|R2JM687C525WR9|          3|           33|         33|   Y|                N|
|R2LWX4TZ67FWPT|          4|           50|         50|   N|                N|
|R29IYHPYD14AGI|          3|           84|         84|   N|                Y|
|R1F2I723WRK5QV|          5|           20|         20|   N|     

In [7]:
# create a new DataFrame to retrieve all rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%
new_df = vote_count_df.filter(vote_count_df["helpful_votes"]/vote_count_df["total_votes"]>=0.5)
new_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| ROP6ITXO8K5V2|          5|           23|         27|   N|                Y|
|R3ND1LVU7AXCVF|          1|           21|         21|   N|                Y|
| R9I5FOLKU99RY|          5|           19|         20|   Y|                N|
|R1QS8AOD6HX3ED|          4|           59|         81|   N|                N|
|R3ED60RC69CCQ6|          5|           22|         23|   Y|                N|
|R2JM687C525WR9|          3|           33|         33|   Y|                N|
|R2LWX4TZ67FWPT|          4|           50|         50|   N|                N|
|R29IYHPYD14AGI|          3|           84|         84|   N|                Y|
|R1F2I723WRK5QV|          5|           20|         20|   N|                Y|
| RRO8C1IVZMD6H|          5|           22|         22|   N|     

In [8]:
# total number of vine reviews
vine_Y_df = new_df.filter("vine == 'Y'")
vine_Y_df.count()

1266

In [9]:
# total number of non-vine reviews
vine_N_df = new_df.filter("vine == 'N'")
vine_N_df.count()

62028

In [21]:
# number of 5-star reviews
five_star_rating = new_df.filter("star_rating == 5")
five_star_rating.count()

30414

In [17]:
# total vine 5-star reviews
vine_five_star = vine_Y_df.filter("star_rating == 5")
vine_five_star.count()

432

In [18]:
# total non-vine 5-star reviews
non_vine_five_star = vine_N_df.filter("star_rating == 5")
non_vine_five_star.count()

29982

In [19]:
# percentage of paid 5-star reviews 
vine_five_star.count() / vine_Y_df.count()

0.3412322274881517

In [22]:
# percentage of unpaid 5-star reviews
non_vine_five_star.count() / vine_N_df.count()

0.4833623524859741